CODE
Подход 2: логистическая регрессия

In [1]:
# Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! 
# Может пригодиться sklearn.preprocessing.StandartScaler.

import numpy as np
import pandas
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold,cross_val_score

features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
y = features['radiant_win'].to_frame()
features.drop(['duration',
               'radiant_win',
               'tower_status_radiant', 
               'tower_status_dire', 
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)
features_clean = features.fillna(0)
features_test_clean = features_test.fillna(0)

rescale_attr = StandardScaler()
features_fin = rescale_attr.fit_transform(features_clean)
Y=np.array(y)


kf = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
#1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с 
# L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для 
# градиентного бустинга. 
from sklearn.linear_model import LogisticRegression

def run_LogReg(features_fin,Y):
    for i in range(-5,5):
        C=10**i
        lr = LogisticRegression(C=C, random_state=241)
        start_time = datetime.datetime.now()
        lr_scores = cross_val_score(lr, features_fin, Y.ravel(), cv=kf, scoring='roc_auc')
        run_time=datetime.datetime.now() - start_time
        print ('Time elapsed:', run_time)
        print (lr_scores)
        lr_qual.append((i,C,run_time.total_seconds(),np.mean(lr_scores)))
    return lr_qual

In [17]:
lr_qual = run_LogReg(features_fin, Y)
lr_qual

Time elapsed: 0:00:03.250486
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
Time elapsed: 0:00:04.499796
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
Time elapsed: 0:00:08.433491
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
Time elapsed: 0:00:11.234739
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
Time elapsed: 0:00:10.852820
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
Time elapsed: 0:00:12.139916
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
Time elapsed: 0:00:11.272202
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
Time elapsed: 0:00:11.656524
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
Time elapsed: 0:00:11.281191
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
Time elapsed: 0:00:11.997719
[0.72165733 0.71369457 0.71438593 0.71539407 0.71639938]


[(-5, 1e-05, 3.368827, 0.695120379847076),
 (-4, 0.0001, 4.462422, 0.7112501143920594),
 (-3, 0.001, 8.185532, 0.7161802463683578),
 (4, 10000, 12.385078, 0.7163062572998156),
 (3, 1000, 12.130183, 0.7163062636530346),
 (2, 100, 18.181951, 0.7163062657792337),
 (1, 10, 13.232057, 0.7163063399602339),
 (0, 1, 11.104389, 0.716306583645544),
 (-1, 0.1, 11.270448, 0.7163100836533356),
 (-2, 0.01, 10.440804, 0.716341462186996),
 (-5, 1e-05, 2.985466, 0.695120379847076),
 (-4, 0.0001, 4.651262, 0.7112501143920594),
 (-3, 0.001, 8.615195, 0.7161802463683578),
 (-2, 0.01, 19.876072, 0.716341462186996),
 (-1, 0.1, 24.528729, 0.7163100836533356),
 (0, 1, 15.178845, 0.716306583645544),
 (1, 10, 14.36206, 0.7163063399602339),
 (2, 100, 16.921301, 0.7163062657792337),
 (3, 1000, 12.653866, 0.7163062636530346),
 (4, 10000, 12.836843, 0.7163062572998156),
 (-5, 1e-05, 3.250486, 0.695120379847076),
 (-4, 0.0001, 4.499796, 0.7112501143920594),
 (-3, 0.001, 8.433491, 0.7161802463683578),
 (-2, 0.01, 11.

In [18]:
# Подберите при этом лучший параметр регуляризации (C).
def bestC(lr_qual):
    lr_qual.sort(key=lambda x:x[3])
    return max(lr_qual, key=lambda x: x[3])[3], max(lr_qual, key=lambda x: x[3])[1]

In [89]:
bestC(lr_qual)

(0.7519018100855892, 0.01)

In [88]:
# Какое наилучшее качество у вас получилось?
all_values = bestC(lr_qual)[0]
all_C = bestC(lr_qual)[1]
print ("best val=%.f, with C=%.2f" % (all_values,all_C))
all_values

best val=1, with C=0.01


0.7519018100855892

 Как оно соотносится с качеством градиентного бустинга? 
 
 0.716341462186996 -> similar to GB with ntrees=200


In [22]:
# Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
time_required = max(lr_qual, key=lambda x: x[3])[2]
time_required #=10.079412

10.440804

GB: datetime.timedelta(0, 321, 256976).total_seconds() = 321.256976

->  30 times faster

In [23]:
#2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, 
#что вряд ли является хорошей идеей.


In [24]:
#Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, 
#d2_hero, ..., d5_hero. Уберите их из выборки,

In [25]:
features_clean_nocat=features_clean.copy()

In [26]:
def remove_cat_attr(features_nocat):
    del features_nocat['lobby_type']
    for n in range(1, 6):
        del features_nocat['r{}_hero'.format(n)]
        del features_nocat['d{}_hero'.format(n)]
    return features_nocat

In [27]:
remove_cat_attr(features_clean_nocat)
features_clean_nocat[:2]

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0


In [28]:
features_fin_nocat = rescale_attr.fit_transform(features_clean_nocat)

In [29]:
lr_qual_nocat = run_LogReg(features_fin_nocat, Y)
lr_qual_nocat


Time elapsed: 0:00:02.442813
[0.69961316 0.69150564 0.69419646 0.69425458 0.69571482]
Time elapsed: 0:00:03.800238
[0.71630684 0.70828717 0.70971489 0.71020968 0.71172337]
Time elapsed: 0:00:07.145027
[0.72152794 0.71360976 0.71445497 0.71521698 0.71636831]
Time elapsed: 0:00:09.135378
[0.7218164  0.71370124 0.71462628 0.71551614 0.71634469]
Time elapsed: 0:00:09.586750
[0.72181205 0.7136378  0.7146095  0.71551012 0.71629945]
Time elapsed: 0:00:10.035968
[0.72181039 0.71363045 0.71460884 0.71551016 0.71629392]
Time elapsed: 0:00:09.863436
[0.72181043 0.71362965 0.71460893 0.71551033 0.71629306]
Time elapsed: 0:00:09.559017
[0.72181067 0.71362945 0.71460889 0.71551027 0.71629322]
Time elapsed: 0:00:09.674702
[0.72181072 0.71362952 0.71460893 0.71551026 0.71629323]
Time elapsed: 0:00:09.557062
[0.72181071 0.71362952 0.71460894 0.71551026 0.71629323]


[(-5, 1e-05, 3.368827, 0.695120379847076),
 (-5, 1e-05, 2.985466, 0.695120379847076),
 (-5, 1e-05, 3.250486, 0.695120379847076),
 (-4, 0.0001, 4.462422, 0.7112501143920594),
 (-4, 0.0001, 4.651262, 0.7112501143920594),
 (-4, 0.0001, 4.499796, 0.7112501143920594),
 (-3, 0.001, 8.185532, 0.7161802463683578),
 (-3, 0.001, 8.615195, 0.7161802463683578),
 (-3, 0.001, 8.433491, 0.7161802463683578),
 (4, 10000, 12.385078, 0.7163062572998156),
 (4, 10000, 12.836843, 0.7163062572998156),
 (4, 10000, 11.997719, 0.7163062572998156),
 (3, 1000, 12.130183, 0.7163062636530346),
 (3, 1000, 12.653866, 0.7163062636530346),
 (3, 1000, 11.281191, 0.7163062636530346),
 (2, 100, 18.181951, 0.7163062657792337),
 (2, 100, 16.921301, 0.7163062657792337),
 (2, 100, 11.656524, 0.7163062657792337),
 (1, 10, 13.232057, 0.7163063399602339),
 (1, 10, 14.36206, 0.7163063399602339),
 (1, 10, 11.272202, 0.7163063399602339),
 (0, 1, 11.104389, 0.716306583645544),
 (0, 1, 15.178845, 0.716306583645544),
 (0, 1, 12.139916

In [87]:
# и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
nocat_values = bestC(lr_qual_nocat)[0]
nocat_C = bestC(lr_qual_nocat)[1]
print ("best val=%.6f, with C=%.2f with dropped category attributes" % (nocat_values,nocat_C))

best val=0.751902, with C=0.01 with dropped category attributes


In [33]:
nocat_values/all_values

1.0000830448450613

 Изменилось ли качество? - no, same
 
Чем вы можете это объяснить? - those attributes do not have any impact on the model

In [34]:
#На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли 
# за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, 
# чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может 
# пригодиться фукнция unique или value_counts).

In [35]:
features_heros=features_clean.copy()
all_heroes1=[]
all_heroes2=[]
for n in range(1, 6):
    all_heroes1.append(features_heros['r{}_hero'.format(n)].unique())
    all_heroes2.append(features_heros['d{}_hero'.format(n)].unique())


In [36]:
features_heros['r1_hero'][2]

33

In [37]:
all_names_of_heroes=[]
for i in range(0,5):
    for j in range(0,len(all_heroes2[i])):
        all_names_of_heroes.append(all_heroes2[i][j])
for i in range(0,5):
    for j in range(0,len(all_heroes1[i])):
        all_names_of_heroes.append(all_heroes1[i][j])        

In [63]:
hero_names = list(set(all_names_of_heroes))
#hero_names

In [39]:
Num_heroes = len(hero_names)
print (Num_heroes)

108


In [56]:
features_clean.shape[0]

97230

In [64]:
#4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев.
#Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, 
#если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. 
#Ниже вы можете найти код, который выполняет данной преобразование. 
# N — количество различных героев в выборке
# Формируем "мешок слов" по героям
def bag_hero_to_num(X):
    X_pick = np.zeros((X.shape[0], Num_heroes))
    for i, match_id in enumerate(X.index):
        for p in range(1,6):
            X_pick[i, np.where(hero_names == X.ix[match_id, 'r%d_hero' % p])] = 1
            X_pick[i, np.where(hero_names == X.ix[match_id, 'd%d_hero' % p])] = -1
    return X_pick


In [70]:
X_pick = bag_hero_to_num(features_clean)
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [115]:
#Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
features_clean_catToNum_base = features_clean_nocat.copy()
features_clean_catToNum_base.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [117]:
features_clean_catToNum = features_clean_catToNum_base.join(pandas.DataFrame(X_pick, index=features_clean_nocat.index))
features_clean_catToNum.head()


,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
features_fin_catToNum = rescale_attr.fit_transform(features_clean_catToNum)
#features_fin_catToNum[:2]

In [119]:
#4 .Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
lr_qual_corCat = run_LogReg(features_fin_catToNum, Y)

Time elapsed: 0:00:05.606984
[0.71907467 0.71117649 0.71512939 0.71288168 0.71586231]
Time elapsed: 0:00:07.789049
[0.74685672 0.74006714 0.74277168 0.74026225 0.74423779]
Time elapsed: 0:00:13.983975
[0.75515333 0.74931124 0.75181648 0.74901026 0.75305284]
Time elapsed: 0:00:20.620703
[0.75521596 0.74985583 0.75218088 0.74951097 0.75308879]
Time elapsed: 0:00:20.392215
[0.75513027 0.74989157 0.75213472 0.74953537 0.75293117]
Time elapsed: 0:00:20.722569
[0.75511969 0.74989604 0.75212742 0.74953884 0.75290574]
Time elapsed: 0:00:20.804055
[0.75511883 0.74989652 0.75212657 0.74953969 0.75290278]
Time elapsed: 0:00:21.294801
[0.75511879 0.74989678 0.75212649 0.74953983 0.75290249]
Time elapsed: 0:00:20.927342
[0.75511885 0.74989672 0.75212658 0.74953981 0.75290238]
Time elapsed: 0:00:21.562589
[0.75511887 0.74989671 0.75212658 0.74953978 0.75290239]


In [120]:
values_with_catToNum = bestC(lr_qual_corCat)[0]
C_with_catToNum = bestC(lr_qual_corCat)[1]
print ("best val=%.4f, with C=%.2f with correccly propagated category attributes" % (values_with_catToNum,C_with_catToNum))

best val=0.7520, with C=0.01 with correccly propagated category attributes


In [121]:
values_with_catToNum/nocat_values

1.0000913403122682

Какое получилось качество? 
0.7519018100855892

Улучшилось ли оно?
+5%

Чем вы можете это объяснить?

Proper use of all atributes: use category attribute as information - not remove, not directly digital

In [83]:
# с помощью лучшей из изученных моделей: update C evaluation on smaller range to find optimum
# C_rough = 0.01, scan again in 0.001 - 0.1
grid_C = np.linspace(0.001,0.1,100)
grid_C

array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018,
       0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027,
       0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035, 0.036,
       0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044, 0.045,
       0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053, 0.054,
       0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062, 0.063,
       0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071, 0.072,
       0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 , 0.081,
       0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089, 0.09 ,
       0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098, 0.099,
       0.1  ])

In [86]:
lr_optimize_qual =[]
#features_fin_catToNum
for i in grid_C:
    C=i
    lr = LogisticRegression(C=C, random_state=241)
    start_time = datetime.datetime.now()
    lr_scores = cross_val_score(lr, features_fin_catToNum, Y.ravel(), cv=kf, scoring='roc_auc')
    run_time=datetime.datetime.now() - start_time
    print (i, run_time, lr_scores)
    lr_optimize_qual.append((i,C,run_time.total_seconds(),np.mean(lr_scores)))

0.001 0:00:15.187795 [0.75503591 0.74925153 0.75165321 0.74902422 0.75306145]
0.002 0:00:16.264617 [0.75518935 0.74960462 0.75194111 0.74934599 0.75325175]
0.003 0:00:17.178558 [0.75518466 0.74969635 0.75200778 0.74943028 0.75324649]
0.004 0:00:17.927311 [0.75516559 0.7497335  0.75202204 0.749468   0.75321367]
0.005 0:00:18.066343 [0.75514967 0.74975284 0.75202147 0.74948709 0.75318775]
0.006 0:00:18.473459 [0.75513352 0.74976621 0.75202368 0.74949457 0.75316259]
0.007 0:00:19.875801 [0.75512131 0.74977477 0.75202086 0.74949987 0.75314054]
0.008 0:00:19.895799 [0.755108   0.74977784 0.75201942 0.74950383 0.75312326]
0.009000000000000001 0:00:19.696915 [0.75509847 0.74978466 0.75201887 0.74950783 0.7531102 ]
0.010000000000000002 0:00:19.854673 [0.75509143 0.74979078 0.75201826 0.74951065 0.75309794]
0.011 0:00:20.811085 [0.75508574 0.74979291 0.75201459 0.74951166 0.75308929]
0.012 0:00:20.121181 [0.7550787  0.74979185 0.752013   0.74951608 0.75307763]
0.013000000000000001 0:00:20.14227

In [93]:
lr_optimize_qual_values = bestC(lr_optimize_qual)[0]
lr_optimize_qual_C = bestC(lr_optimize_qual)[1]
print ("best val=%.6f, with C=%.5f with converted category attributes" % (lr_optimize_qual_values,lr_optimize_qual_C))
#best val=0.751921, with C=0.00400 with dropped category attributes

best val=0.751921, with C=0.00400 with dropped category attributes


In [ ]:
#Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей
#(лучшей с точки зрения AUC-ROC на кросс-валидации). 

In [104]:
features_clean_catToNum.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
C=0.004
lrfin = LogisticRegression(C=C, random_state=241)
lrfin.fit(features_fin_catToNum, Y.ravel())

LogisticRegression(C=0.004, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [123]:
# Prepate test dataset:
# 1. take test data with cleaned missing values
features_test_clean_nocat = features_test_clean.copy()

# 2. remove category attributes
remove_cat_attr(features_test_clean_nocat)

# 3. create attributes based on category - heroes type
X_pick_test = bag_hero_to_num(features_test_clean)

# 4. add newly created attributes to initial test matrix without categories
features_test_clean_catToNum = features_test_clean_nocat.join(pandas.DataFrame(X_pick_test, index=features_test_clean.index))

# 5. normalize attributes in final test dataset
features_test_fin1 = rescale_attr.fit_transform(features_test_clean_catToNum)



In [128]:
rad_win_pred = lrfin.predict_proba(features_test_fin1)
rad_win_pred

array([[0.18063632, 0.81936368],
       [0.24736102, 0.75263898],
       [0.80879367, 0.19120633],
       ...,
       [0.76255588, 0.23744412],
       [0.37022597, 0.62977403],
       [0.56642917, 0.43357083]])

In [129]:
#Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).
print("min = %.3f, max= %.3f" % (rad_win_pred.min(), rad_win_pred.max()))


min = 0.004, max= 0.996


Report part 2:
Q1. Какое качество получилось у логистической регрессии над всеми исходными признаками? 1.2: Как оно соотносится с качеством градиентного бустинга? 1.3: Чем вы можете объяснить эту разницу? 1.4: Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

A1.1: 0.71634 (all_values в коде);
A1.2: соответствует качеству очень глубокого дерева (0.71634 GB ntrees=200)
A1.3: Линейная регрессия оптимально проводит разделяющую гиперплоскость для объектов двух классов. Градиетный бустинг хуже подходит для разреженных данных
A1.4: время LR~10s (time_required в коду), у бустинга ~320s -> в 30 раз быстрее

Q2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? 2.2: Чем вы можете объяснить это изменение?

A2.1: Почти не меняется : ( в коде: значение= values_with_catToNum, отношение= values_with_catToNum/nocat_values=1.0000830448450613)

Q3. Сколько различных идентификаторов героев существует в данной игре?
A3: 108

Q4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
A. 

Q5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
min = 0.004, max= 0.996